In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from ekf import ExtendedKalmanFilter

# Simulated system: 1D constant velocity
dt = 1.0  # time step

# Define the nonlinear state transition function f(x, u)
def f(x, u=None):
    F = np.array([[1, dt],
                  [0, 1]])
    return F @ x

# Define the nonlinear measurement function h(x)
def h(x):
    return np.array([x[0]])  # measuring position only

# Jacobian of f w.r.t x
def F_jacobian(x, u=None):
    return np.array([[1, dt],
                     [0, 1]])

# Jacobian of h w.r.t x
def H_jacobian(x):
    return np.array([[1, 0]])

# Initial state
x0 = np.array([0.0, 1.0])  # starting at position 0 with velocity 1 m/s

# Covariances
P0 = np.eye(2) * 1.0
Q = np.array([[0.1, 0.0],
              [0.0, 0.1]])  # process noise
R = np.array([[1.0]])        # measurement noise

# Create EKF instance
kf = ExtendedKalmanFilter(f, h, F_jacobian, H_jacobian, Q, R, P0, x0)

# Simulate motion and noisy measurements
num_steps = 20
true_states = []
measurements = []
estimates = []

true_x = x0.copy()

for i in range(num_steps):
    # Simulate true motion
    true_x = f(true_x)
    true_states.append(true_x.copy())

    # Generate noisy measurement of position
    z = h(true_x) + np.random.normal(0, np.sqrt(R[0, 0]), size=(1,))
    measurements.append(z)

    # EKF predict and update
    kf.predict()
    x_est = kf.update(z)
    estimates.append(x_est)

# Convert results to numpy arrays
true_states = np.array(true_states)
measurements = np.array(measurements).flatten()
estimates = np.array(estimates)

# Plot results
plt.figure(figsize=(10, 5))
plt.plot(true_states[:, 0], label='True Position')
plt.plot(measurements, 'o', label='Measured Position', alpha=0.5)
plt.plot(estimates[:, 0], label='EKF Estimated Position')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Position')
plt.title('EKF Test - 1D Constant Velocity')
plt.grid(True)
plt.show()